In [1]:
import os

import numpy as np
from tqdm import trange
from tinygrad.nn import Conv2d, Linear, optim
from tinygrad.tensor import Tensor
from extra.training import train, evaluate
from datasets import fetch_mnist

ops_triton not available No module named 'pycuda'


In [2]:
os.environ["GPU"] = "1"
os.environ["CL_DEVICE"] = "1"
np.random.seed(1337)

In [3]:
class ConvNet:

    def __init__(self):
        self.conv1 = Conv2d(1, 32, 5)
        self.conv2 = Conv2d(32, 64, 5)
        self.fc1 = Linear(4 * 4 * 64, 128)
        self.fc2 = Linear(128, 10)
    
    def __call__(self, x: Tensor) -> Tensor:
        x = self.conv1(x).relu().max_pool2d(kernel_size=(2, 2))
        x = self.conv2(x).relu().max_pool2d(kernel_size=(2, 2))
        x = x.reshape(-1, 4 * 4 * 64)
        x = self.fc1(x).relu()
        x = self.fc2(x).logsoftmax()
        return x

In [4]:
X_train, Y_train, X_test, Y_test = fetch_mnist()
X_train = X_train.reshape(-1, 1, 28, 28) / 255.0
X_test = X_test.reshape(-1, 1, 28, 28) / 255.0
model = ConvNet()
optimizer = optim.Adam(optim.get_parameters(model), lr=0.01)

In [5]:
train(model, X_train, Y_train, optimizer, 50, BS=64)

loss 0.06 accuracy 0.98: 100%|██████████| 50/50 [00:24<00:00,  2.03it/s]


In [6]:
evaluate(model, X_test, Y_test);

100%|██████████| 79/79 [00:19<00:00,  4.09it/s]

test set accuracy is 0.952300
